In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.layers import Input, Dense, Flatten, Reshape, Conv2D,Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.python.keras.models import Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics,preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
import seaborn as sns # data visualization

In [ ]:
train_df = pd.read_csv('//kaggle//input//digit-recognizer//train.csv')
test_df = pd.read_csv('//kaggle//input//digit-recognizer//test.csv')


train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
labels = train_df.iloc[:,0]
nb_labels = labels.max() + 1

X_train = train_df.iloc[:,1:]


print(train_df.columns)

In [ ]:
fig, ax = plt.subplots()
image = test_df.values
for i in range(0,3):
    im = plt.imshow(image[i].reshape(28,28), cmap='gray')
    plt.show()


In [ ]:
# Connect input, intermediate, and output layers using the Keras functional API
i = Input(shape=(28,28,1))
x = Conv2D(8, kernel_size=(3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(i)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(16, kernel_size=(3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),)(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(24, kernel_size=(3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),)(x)
x = Flatten()(x)
x = Dense(20, activation='relu')(x)
o = Dense(nb_labels, activation='softmax')(x)

# Create Model
cnn = Model(i, o, name='CNN')

In [ ]:
cnn.summary()

In [ ]:
cnn.compile(optimizer='adam', 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

In [ ]:
X = X_train.values.reshape(X_train.shape[0],28,28,1)
print(X.shape)
print(labels.shape)


In [ ]:
history_cnn = cnn.fit(x=X, y=labels, epochs=20, validation_split=0.2, batch_size=32)

In [ ]:
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y = test_df.values.reshape(test_df.shape[0],28,28,1)
prediction  = cnn.predict(y)


In [ ]:
predictions = prediction.argmax(axis =1)
ids = np.arange(1,len(predictions)+1)
output = pd.DataFrame({'ImageId': ids, 'Label': predictions})
print(output)

In [ ]:
output.to_csv('submission.csv', index=False)